In [7]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [8]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [9]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [11]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "fCo90Tj0aFUYJbZivnT15CwEK"
    consumer_secret = "csvQnYSOloE51u8PYapT5L1fy1rjSu2jmGweSEzgsTWt5MlgLI"
    access_token = "1517633626409644032-swmNuV04Pbc9WHp1NBP48jt1yZ1X73"
    access_token_secret = "KlTmlFVgq0DXR00BO8jr4bSl9aMe20x6BWkF0xXnlEyDl"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [117.17427453, 5.58100332277, 126.537423944, 18.5052273625]  # Contiguous US

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")
    stream.filter(track=['coronavirus'])

{'id': 1517676223945601024, 'username': '_shurik3n_', 'created_at': 'Sat Apr 23 01:26:23 +0000 2022', 'lng': 121.010803, 'lat': 14.483414, 'text': "we sing along with songs we're not supposed to sing just because we're sad 😞"}
{'id': 1517676270368456704, 'username': '6ftarkie', 'created_at': 'Sat Apr 23 01:26:34 +0000 2022', 'lng': 121.03114149999999, 'lat': 14.710893500000001, 'text': "i cant attend lang din po kasi for today's rally...HAPPY BIRTHDAY DAY VP LENI!!!! 💕💕💕💕THANK YOU SO MUCH… https://t.co/8ndLgrSEYr"}
{'id': 1517676273803624450, 'username': 'DonaldDakz', 'created_at': 'Sat Apr 23 01:26:35 +0000 2022', 'lng': 121.082345, 'lat': 14.583306499999999, 'text': 'Birthday? Dapat may Party Hats heheheSee you later and Happy Birthday @lenirobredo !#ArawNa10Ito #PasayIsPink… https://t.co/9K5gxXq77I'}
{'id': 1517676297367212032, 'username': 'mariagrasya_', 'created_at': 'Sat Apr 23 01:26:41 +0000 2022', 'lng': 121.108852, 'lat': 14.188623, 'text': 'Grabe goosebumps'}
{'id': 151767639

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
